# Quickstart: Classic RAG in Azure AI Search

This quickstart provides a query for RAG scenarios. It's based on the [classic RAG pattern](https://learn.microsoft.com/azure/search/search-agentic-retrieval-concept) in Azure AI Search. This pattern uses a chat completion model to provide a response to a query posed to your content on Azure AI Search.

We took a few shortcuts to keep the exercise basic and focused on query definitions:

- We use the [**hotels-sample-index**](https://review.learn.microsoft.com/azure/search/search-get-started-portal), which is created in other quickstarts.

- We omit vectors so that we can skip chunking and embedding. The index contains plain text. A non-vector index isn't ideal for RAG patterns, but it makes for a simpler example.

## Prerequisites

- [Azure OpenAI](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource)

  - [Choose a region](https://learn.microsoft.com/azure/ai-services/openai/concepts/models?tabs=global-standard%2Cstandard-chat-completions#global-standard-model-availability) that supports the chat completion model you want to use (gpt-4o, gpt-4o-mini, or equivalent model). 
  - [Deploy the chat completion model](https://learn.microsoft.com/azure/ai-studio/how-to/deploy-models-openai) in Microsoft Foundry or [use another approach](https://learn.microsoft.com/azure/ai-services/openai/how-to/working-with-models?tabs=powershell).


- [Azure AI Search](https://learn.microsoft.com/azure/search/search-create-service-portal)

  - Basic tier or higher is recommended.
  - Enable semantic ranking.
  - Enable role-based access control.
  - Enable a system identity for Azure AI Search.
  
+ [Visual Studio Code](https://code.visualstudio.com/download) with the [Python extension](https://marketplace.visualstudio.com/items?itemName=ms-python.python) and [Jupyter package](https://pypi.org/project/jupyter/).

Make sure you know the name of the deployed model, and have the endpoints for both Azure resources at hand. You will provide this information in the steps that follow.

## Create a virtual environment

Create a virtual environment so that you can install the dependencies in isolation.

1. In Visual Studio Code, open the folder containing Quickstart-RAG.ipynb.

1. Press Ctrl-shift-P to open the command palette, search for "Python: Create Environment", and then select `Venv` to create a virtual environment in the current workspace.

1. Select Quickstart-RAG\requirements.txt for the dependencies.

It takes several minutes to create the environment. When the environment is ready, continue to the next step.
  
## Sign in to Azure

You're using Microsoft Entra ID and role assignments for the connection. Make sure you're logged in to the same tenant and subscription as Azure AI Search and Azure OpenAI. You can use the Azure CLI on the command line to show current properties, change properties, and to log in. For more information, see [Connect without keys](https://learn.microsoft.com/azure/search/search-get-started-rbac). 

Run each of the following commands in sequence.

```
az account show

az account set --subscription <PUT YOUR SUBSCRIPTION ID HERE>

az login --tenant <PUT YOUR TENANT ID HERE>
```

You should now be logged in to Azure from your local device.

## Configure access

Requests to the search endpoint must be authenticated and authorized. You can use API keys or roles for this task. Keys are easier to start with, but roles are more secure. This quickstart assumes roles.

You're setting up two clients, so you need permissions on both resources.

Azure AI Search is receiving the query request from your local system. Assign yourself the **Search Index Data Reader** role assignment if the hotels sample index already exists. If it doesn't exist, assign yourself **Search Service Contributor** and **Search Index Data Contributor** roles so that you can create and load the index.

Azure OpenAI is receiving the query and the search results from your local system. Assign yourself the **Cognitive Services OpenAI User** role on Azure OpenAI.

1. Sign in to the [Azure portal](https://portal.azure.com).

1. Configure Azure AI Search for role-based access:

    1. In the Azure portal, find your Azure AI Search service.

    1. On the left menu, select **Settings** > **Keys**, and then select either **Role-based access control** or **Both**.

1. Assign roles:

    1. On the left menu, select **Access control (IAM)**.

    1. On Azure AI Search, select these roles to create, load, and query a search index, and then assign them to your Microsoft Entra ID user identity:

       - **Search Index Data Contributor**
       - **Search Service Contributor**

    1. On Azure OpenAI, select **Access control (IAM)** to assign this role to yourself on Azure OpenAI:

       - **Cognitive Services OpenAI User**

It can take several minutes for permissions to take effect.

## Get service endpoints and tokens

In the remaining sections, you set up API calls to Azure OpenAI and Azure AI Search. Get the service endpoints and tokens so that you can provide them as variables in your code.

1. Sign in to the [Azure portal](https://portal.azure.com).

1. [Find your search service](https://portal.azure.com/#blade/HubsExtension/BrowseResourceBlade/resourceType/Microsoft.Search%2FsearchServices).

1. On the **Overview** home page, copy the URL. An example endpoint might look like `https://example.search.windows.net`. 

1. [Find your Azure OpenAI service](https://portal.azure.com/#blade/HubsExtension/BrowseResourceBlade/resourceType/Microsoft.CognitiveServices%2Faccounts).

1. On the **Overview** home page, select the link to view the endpoints. Copy the URL. An example endpoint might look like `https://example.openai.azure.com/`.

1. Get personal access tokens from the Azure CLI on a command prompt. Here are the commands for each resource:

   - `az account get-access-token --resource https://search.azure.com --query "accessToken" -o tsv`
   - `az account get-access-token --resource https://cognitiveservices.azure.com --query "accessToken" -o tsv`

## Prepare the sample index

This quickstart assumes the hotels-sample-index, which you can create using [this quickstart](https://learn.microsoft.com/azure/search/search-get-started-portal).

Once the index exists, use the **Edit JSON** action in the Azure portal to add this semantic configuration:

```json
"semantic":{
    "defaultConfiguration":"semantic-config",
    "configurations":[
        {
            "name":"semantic-config",
            "prioritizedFields":{
            "titleField":{
                "fieldName":"HotelName"
            },
            "prioritizedContentFields":[
                {
                    "fieldName":"Description"
                }
            ],
            "prioritizedKeywordsFields":[
                {
                    "fieldName":"Category"
                },
                {
                    "fieldName":"Tags"
                }
            ]
            }
        }
    ]
},
```


Now that you have your Azure resources, an index, and model in place, you can run the script to chat with the index.

## Run the code

In [1]:
# Package install for quickstart
! pip install -r requirements.txt --quiet

In [2]:
# Set endpoints and deployment model (provide the name of the deployment)
AZURE_SEARCH_SERVICE: str = "PUT YOUR SEARCH SERVICE ENDPOINT HERE"
AZURE_OPENAI_ACCOUNT: str = "PUT YOUR AZURE OPENAI ENDPOINT HERE"
AZURE_DEPLOYMENT_MODEL: str = "gpt-4o"

## Basic RAG Query

The following code demonstrates how to set up a query on string fields and string collections in a search index, and how to send the results to a chat completion model. The response returned to the user is from the chat completion model.

In [ ]:
# Set up the query for generating responses
from azure.identity import DefaultAzureCredential
from azure.identity import get_bearer_token_provider
from azure.search.documents import SearchClient
from openai import AzureOpenAI

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
openai_client = AzureOpenAI(
    api_version="2024-06-01",
    azure_endpoint=AZURE_OPENAI_ACCOUNT,
    azure_ad_token_provider=token_provider
)

search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE,
    index_name="hotels-sample-index",
    credential=credential
)

# This prompt provides instructions to the model
GROUNDED_PROMPT="""
You are a friendly assistant that recommends hotels based on activities and amenities.
Answer the query using only the sources provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

# Query is the question being asked. It's sent to the search engine and the LLM.
query="Can you recommend a few hotels with complimentary breakfast?"

# Search results are created by the search client.
# Search results are composed of the top 5 results and the fields selected from the search index.
# Search results include the top 5 matches to your query.
search_results = search_client.search(
    search_text=query,
    top=5,
    select="Description,HotelName,Tags",
    query_type="semantic"
)
sources_formatted = "\n".join([f'{document["HotelName"]}:{document["Description"]}:{document["Tags"]}' for document in search_results])

# Send the search results and the query to the LLM to generate a response based on the prompt.
response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=AZURE_DEPLOYMENT_MODEL
)

# Here is the response from the chat model.
print(response.choices[0].message.content)

Output is from Azure OpenAI, and it consists of recommendations for several hotels. Here's an example of what the output might look like:

```
Sure! Here are a few hotels that offer complimentary breakfast:

- **Head Wind Resort**
- Complimentary continental breakfast in the lobby
- Free Wi-Fi throughout the hotel

- **Double Sanctuary Resort**
- Continental breakfast included

- **White Mountain Lodge & Suites**
- Continental breakfast available

- **Swan Bird Lake Inn**
- Continental-style breakfast each morning with a variety of food and drinks 
    such as caramel cinnamon rolls, coffee, orange juice, milk, cereal, 
    instant oatmeal, bagels, and muffins
```
If you get an authorization error instead of results:

+ If you just enabled role assignments, wait a few minutes and try again. It can take several minutes for role assignments to become operational.

+ Make sure you [enabled RBAC](https://learn.microsoft.com/azure/search/search-security-enable-roles?tabs=config-svc-portal%2Cdisable-keys-portal) on Azure AI Search. An HttpStatusMessage of **Forbidden** is an indicator that RBAC isn't enabled.

+ Recheck role assignments if you get a 401 error. You should also review the steps in [Connect without keys](https://learn.microsoft.com/azure/search/search-get-started-rbac).

+ Check firewall settings. This quickstart assumes public network access. If you have a firewall, you need to add rules to allow inbound requests from your device and for service-to-service connections. For more information, see [Configure network access](https://learn.microsoft.com/azure/search/service-configure-firewall).

+ If you get a *Resource not found* error message, check the resource URIs and make sure the API version on the chat model is valid.

## Complex RAG Query

Azure AI Search supports [complex types](https://learn.microsoft.com/azure/search-howto-complex-data-types) for nested JSON structures. In the hotels-sample-index, `Address` is an example of a complex type, consisting of `Address.StreetAddress`, `Address.City`, `Address.StateProvince`, `Address.PostalCode`, and `Address.Country`. The index also has complex collection of `Rooms` for each hotel.

If your index has complex types, your query can provide those fields if you first convert the search results output to JSON, and then pass the JSON to the chat model. The following example adds complex types to the request. The formatting instructions include a JSON specification.

In [ ]:
import json

# Query is the question being asked. It's sent to the search engine and the LLM.
query="Can you recommend a few hotels that offer complimentary breakfast? Tell me their description, address, tags, and the rate for one room they have which sleep 4 people."

# Set up the search results and the chat thread.
# Retrieve the selected fields from the search index related to the question.
selected_fields = ["HotelName","Description","Address","Rooms","Tags"]
search_results = search_client.search(
    search_text=query,
    top=5,
    select=selected_fields,
    query_type="semantic"
)
sources_filtered = [{field: result[field] for field in selected_fields} for result in search_results]
sources_formatted = "\n".join([json.dumps(source) for source in sources_filtered])

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=AZURE_DEPLOYMENT_MODEL
)

print(response.choices[0].message.content)

Output is from Azure OpenAI, and it adds content from complex types.

```
Here are a few hotels that offer complimentary breakfast and have rooms that sleep 4 people:

1. **Head Wind Resort**
   - **Description:** The best of old town hospitality combined with views of the river and 
   cool breezes off the prairie. Enjoy a complimentary continental breakfast in the lobby, 
   and free Wi-Fi throughout the hotel.
   - **Address:** 7633 E 63rd Pl, Tulsa, OK 74133, USA
   - **Tags:** Coffee in lobby, free Wi-Fi, view
   - **Room for 4:** Suite, 2 Queen Beds (Amenities) - $254.99

2. **Double Sanctuary Resort**
   - **Description:** 5-star Luxury Hotel - Biggest Rooms in the city. #1 Hotel in the area 
   listed by Traveler magazine. Free WiFi, Flexible check in/out, Fitness Center & espresso 
   in room. Offers continental breakfast.
   - **Address:** 2211 Elliott Ave, Seattle, WA 98121, USA
   - **Tags:** View, pool, restaurant, bar, continental breakfast
   - **Room for 4:** Suite, 2 Queen Beds (Amenities) - $254.99

3. **Swan Bird Lake Inn**
   - **Description:** Continental-style breakfast featuring a variety of food and drinks. 
   Locally made caramel cinnamon rolls are a favorite.
   - **Address:** 1 Memorial Dr, Cambridge, MA 02142, USA
   - **Tags:** Continental breakfast, free Wi-Fi, 24-hour front desk service
   - **Room for 4:** Budget Room, 2 Queen Beds (City View) - $85.99

4. **Gastronomic Landscape Hotel**
   - **Description:** Known for its culinary excellence under the management of William Dough, 
   offers continental breakfast.
   - **Address:** 3393 Peachtree Rd, Atlanta, GA 30326, USA
   - **Tags:** Restaurant, bar, continental breakfast
   - **Room for 4:** Budget Room, 2 Queen Beds (Amenities) - $66.99
...
   - **Tags:** Pool, continental breakfast, free parking
   - **Room for 4:** Budget Room, 2 Queen Beds (Amenities) - $60.99

Enjoy your stay! Let me know if you need any more information.
```

## Troubleshooting errors

To debug authentication errors, insert the following code before the step that calls the search engine and the LLM.

```python
import sys
import logging # Set the logging level for all azure-storage-* libraries
logger = logging.getLogger('azure.identity') 
logger.setLevel(logging.DEBUG)

handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('[%(levelname)s %(name)s] %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
```

Rerun the query script. You should now get INFO and DEBUG statements in the output that provide more detail about the issue.

If you see output messages related to ManagedIdentityCredential and token acquisition failures, it could be that you have multiple tenants, and your Azure sign-in is using a tenant that doesn't have your search service. To get your tenant ID, search the Azure portal for "tenant properties" or run `az login tenant list`.

Once you have your tenant ID, run `az login --tenant <YOUR-TENANT-ID>` at a command prompt, and then rerun the script.